In [60]:
%reload_ext autoreload
%autoreload 2

import torch
import numpy as np  
from torch import nn, tensor, Tensor
from torch.nn import functional
import sys, os
sys.path.append("../..")
from hwr_utils.stroke_recovery import relativefy_batch_torch, test_conv_weight, conv_weight

test_conv_weight()

In [61]:
gt_length = 20
rel_x = np.random.randint(0,10,20)
#rel_x = np.array(range(0,gt_length))
rel_y = np.random.randint(0,10,20)
#rel_y = np.array(range(0,gt_length))
start = np.random.randint(0,2,gt_length)

gt = np.c_[rel_x, rel_y, start][np.newaxis] # BATCH, WIDTH, HEIGHT/VOCAB
pred = gt.copy()
pred[:,7,0] = 12

# Batch it up
# gt = np.r_[gt, pred]
# print(gt.shape)

pred = Tensor(pred) # relative
gt = Tensor(gt)
gt_rel = Tensor(gt)
gt = torch.cumsum(gt, dim=1) # abs


In [59]:
#m = nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
KERNEL_LENGTH = 9
KERNEL = Tensor(np.cumsum([.1]*KERNEL_LENGTH)[np.newaxis].transpose(1,0)).repeat(1, 1, 1, 1)
INVERSE_KERNEL = torch.flip(KERNEL, dims=(2,))

def conv_weight(gt_abs, pred_rel, gt_rel=None, indices=slice(0,None)):
    """ BATCH, WIDTH (GT LENGTH), HEIGHT (VOCAB SIZE)
        INDICES MUST BE SLICE/LIST
    """
    width = gt_abs.shape[1]

    cumsum = torch.zeros(*gt_abs.shape)
    cumsum[:, KERNEL_LENGTH:, indices] = gt_abs[:, :width - KERNEL_LENGTH, indices]  # BATCH, WIDTH, VOCAB

    if gt_rel is None:
        gt_rel = relativefy_batch_torch(gt_abs.detach().clone(), indices=indices)

    # Add channel dimension
    gt_rel_exp = gt_rel.unsqueeze(1)[:,:,:,indices]  # BATCH, CHANNEL, WIDTH, VOCAB
    pred_rel_exp = pred_rel.unsqueeze(1)[:,:,:,indices]

    # Functionary way
    gt_rel[:,:,indices] = functional.conv2d(gt_rel_exp, INVERSE_KERNEL, padding=[KERNEL_LENGTH - 1, 0]).squeeze(1)[:,:width]
    pred_rel[:,:,indices] = functional.conv2d(pred_rel_exp, KERNEL, padding=[KERNEL_LENGTH - 1, 0]).squeeze(1)[:,:width] + gt_rel[:,:,indices] + cumsum[:,:,indices]
    return pred_rel

# Anything with itself should be equivalent to a cumulative sum
x = conv_weight(gt,gt_rel.clone())
np.testing.assert_almost_equal(x.numpy(), gt, decimal=5)

# Should get back on track
x = conv_weight(gt,pred.clone())
np.testing.assert_almost_equal(x[:,-4:].numpy(), gt[:,-4:], decimal=5)
np.testing.assert_almost_equal(x[:,0:6].numpy(), gt[:,0:6], decimal=5)

# With batching
gt2 = Tensor(np.r_[gt,gt])
gt2_rel = Tensor(np.r_[gt_rel.clone(),gt_rel.clone()]) # pred is relative
x = conv_weight(gt2,gt2_rel)
np.testing.assert_almost_equal(x.numpy(), gt2, decimal=5)

# With batching + index
gt2 = Tensor(np.r_[gt,gt])
gt2_rel = Tensor(np.r_[gt_rel.clone(),gt_rel.clone()]) # pred is relative
x = conv_weight(gt2,gt2_rel, indices=[1,])
np.testing.assert_almost_equal(x.numpy()[:,:,1], gt2[:,:,1], decimal=5)
np.testing.assert_almost_equal(x.numpy()[:,:,0], gt2_rel[:,:,0], decimal=5)


tensor([[[  9.,   9.,   1.],
         [ 10.,  16.,   2.],
         [ 12.,  17.,   2.],
         [ 12.,  19.,   3.],
         [ 16.,  26.,   4.],
         [ 16.,  30.,   4.],
         [ 22.,  36.,   4.],
         [ 26.,  45.,   4.],
         [ 32.,  52.,   5.],
         [ 34.,  54.,   5.],
         [ 42.,  63.,   5.],
         [ 50.,  67.,   5.],
         [ 50.,  75.,   5.],
         [ 54.,  80.,   6.],
         [ 59.,  87.,   7.],
         [ 62.,  96.,   8.],
         [ 68., 103.,   9.],
         [ 73., 107.,  10.],
         [ 73., 114.,  10.],
         [ 76., 122.,  10.]],

        [[  9.,   9.,   1.],
         [ 10.,  16.,   2.],
         [ 12.,  17.,   2.],
         [ 12.,  19.,   3.],
         [ 16.,  26.,   4.],
         [ 16.,  30.,   4.],
         [ 22.,  36.,   4.],
         [ 26.,  45.,   4.],
         [ 32.,  52.,   5.],
         [ 34.,  54.,   5.],
         [ 42.,  63.,   5.],
         [ 50.,  67.,   5.],
         [ 50.,  75.,   5.],
         [ 54.,  80.,   6.],
         [ 5

In [14]:
print(x)

tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])

In [82]:
cumsum[:,:,:,kernel_length:]

array([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]])

20

In [27]:
()


[1, 20, 1]

In [16]:
[*gt.shape].insert(1,1)